# Import Important Modul

In [27]:
import re
import requests
import pdfplumber
import pandas as pd
from io import BytesIO

## Create Load & Preprocessing Function 

In [23]:
def download_pdf(url):
    response = requests.get(url)
    response.raise_for_status()
    return BytesIO(response.content)

def extract_lines_from_pdf(pdf_file):
    lines = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                page_lines = page_text.split('\n')
                lines.extend(page_lines)
    return lines

def clean_line(line):
    line = re.sub(r'\s+', ' ', line).strip()
    line = remove_whitespace_in_numbers(line)
    line = remove_dash_slash(line)
    line = convert_parentheses_to_negative(line)
    line = remove_digit_parenthesis_after_text(line)
    line = remove_space_between_antar_kantor(line)
    line = remove_blank_parenthesis(line)
    return line

def remove_whitespace_in_numbers(line):
    line = re.sub(r'(\d)\s+(\d)', r'\1\2', line)
    line = re.sub(r'(\d)\s+,', r'\1,', line)
    line = re.sub(r'\(\s*(\d)', r'(\1', line)
    line = re.sub(r'(\d)\s*\)', r'\1)', line)
    return line

def convert_parentheses_to_negative(line):
    return re.sub(r'\(([\d.,]+)\)', r'-\1', line)

def remove_dash_slash(value):
    value = value.replace('-/-', '')
    return re.sub(r'\s+', ' ', value).strip()

def remove_digit_parenthesis_after_text(text):
    return re.sub(r'(\D)\d\)', r'\1', text)

def remove_space_between_antar_kantor(text):
    return re.sub(r'\bantar\s+kantor\b', 'antarkantor', text)

def remove_blank_parenthesis(text):
    return re.sub(r"(\w+\.\s+[^\(]+)\s*\(\)\s+([\d.,]+)", r"\1\2", text).strip()

def merge_multiline_text_with_keywords(lines, keywords=None):
    if keywords is None:
        keywords = []

    merged_lines = []
    temp_line = ""

    for i, line in enumerate(lines):
        stripped_line = line.strip()

        # If line ends with a dash (-), treat it as a separate line
        if stripped_line.endswith("-"):
            if temp_line:
                merged_lines.append(temp_line.strip())
                temp_line = ""
            merged_lines.append(stripped_line)
        elif re.search(r"\d$", stripped_line):  # Ends with a number
            if temp_line:
                merged_lines.append(temp_line.strip())
            temp_line = stripped_line
        else:
            # Current line is treated as the beginning of the next
            if i + 1 < len(lines):
                lines[i + 1] = f"{stripped_line} {lines[i + 1]}".strip()
            else:
                if temp_line:
                    merged_lines.append(temp_line.strip())
                merged_lines.append(stripped_line)
                temp_line = ""
    if temp_line:
        merged_lines.append(temp_line.strip())

    return merged_lines

def process_pdf_from_url(url):
    pdf_file = download_pdf(url)
    lines = extract_lines_from_pdf(pdf_file)
    cleaned_lines = [clean_line(line) for line in lines]
    merged_lines = merge_multiline_text_with_keywords(cleaned_lines)
    return merged_lines

def open_pdf_from_filepath(filepath):
    with open(filepath, 'rb') as file:
        return BytesIO(file.read())

def process_pdf_from_filepath(filepath):
    pdf_file = open_pdf_from_filepath(filepath)
    lines = extract_lines_from_pdf(pdf_file)
    cleaned_lines = [clean_line(line) for line in lines]
    merged_lines = merge_multiline_text_with_keywords(cleaned_lines)
    return merged_lines

## Create def pattern specified amount

In [28]:
def extract_account_amount(line):
    patterns = {
        "Kas": r"(?i)\bKas\b\s+([\d.,]+)",
        "Penempatan pada Bank Indonesia": r"(?i)\bPenempatan pada Bank Indonesia\b\s+([\d.,]+)",
        "Penempatan pada bank lain": r"(?i)\bPenempatan pada bank lain\b\s+([\d.,]+)",
        "Tagihan spot dan derivatif/forward": r"(?i)\bTagihan spot dan derivatif/forward\b\s+([\d.,]+)",
        "Surat berharga yang dimiliki": r"(?i)\bSurat berharga yang dimiliki\b\s+([\d.,]+)",
        "Surat berharga yang dijual dengan janji dibeli kembali (repo)":r"(?i)^\d+\.\s*Surat berharga yang dijual dengan janji dibeli kembali \(repo\)\s+([\d.,]+)",
        "Tagihan atas surat berharga yang dibeli dengan janji dijual kembali (reverse repo)": r"(?i)\bdijual kembali\b\s+\(reverse repo\)\s+([\d.,]+)",
        "Tagihan akseptasi": r"(?i)\bTagihan akseptasi\b\s+([\d.,]+)",
        "Kredit dan pembiayaan yang diberikan": r"(?i)\b(?:Kredit(?: dan pembiayaan)? yang diberikan)\b\s+([\d.,]+)",
        "Pembiayaan syariah": r"(?i)\bPembiayaan syariah\b\s*(?:(?:\(\d+\)\s*)|(?:\[\d+\]\s*)|(?:1\))\s*)?([\d.,-]+)",
        "Penyertaan Modal": r"(?i)\bPenyertaan Modal\b\s+([\d.,]+)",
        "Aset keuangan lainnya": r"(?i)\bAset keuangan lainnya\b\s+([\d.,]+)",
        "Cadangan kerugian penurunan nilai aset keuangan -/-": r"(?i)\b\ dont modify this codeb\s+\-\-\s+(\([\d.,]+\))",
        "a. Surat berharga yang dimiliki": r"(?i)\ba\. Surat berharga yang dimiliki\b\s+(-?[\d.,]+)",
        "b. Kredit yang diberikan dan pembiayaan syariah": r"(?i)\bb\.\s*Kredit yang diberikan dan pembiayaan syariah(?:\d\))?\s*(-?[\d.,]+)",
        "c. Lainnya": r"(?i)\bc\.\s*Lainnya\s*(-?[\d.,]+)",
        "Aset tidak berwujud": r"(?i)\bAset tidak berwujud\b\s+([\d.,()]+)",
        "Akumulasi amortisasi aset tidak berwujud": r"(?i)\bAkumulasi amortisasi aset tidak berwujud\b\s+(-?[\d.,]+)",
        "Aset tetap dan inventaris": r"(?i)\bAset tetap dan inventaris\b\s+([\d.,]+)",
        "Akumulasi penyusutan aset tetap dan inventaris -/-": r"(?i)\bAkumulasi penyusutan aset tetap dan inventaris\b\s+(-?[\d.,]+)",
        "Aset non produktif": r"(?i)\bAset non produktif\b\s+([\d.,()]+)",
        "a. Properti terbengkalai": r"(?i)\ba\.\s*Properti terbengkalai\b\s*(-?[\d.,]+)?",
        "b. Aset yang diambil alih": r"(?i)\bb\.\s*(?:Agunan|Aset)\s*yang diambil alih\b\s*(-?[\d.,]+)?",
        "c. Rekening tunda": r"(?i)\bc\.\s*Rekening tunda\b\s*(-?[\d.,]+)?",
        "d. Aset antarkantor": r"(?i)\bd\.\s*Aset antarkantor\b\s*([\d.,]+)",
        "Aset lainnya": r"(?i)\bAset lainnya\b\s+([\d.,()]+)",
        "Total Aset": r"(?i)\b dont modify this code\b\s+([\d.,]+)",
        "Giro": r"(?i)\bGiro\b\s+([\d.,\-]+)",
        "Tabungan": r"(?i)\bTabungan\b\s+([\d.,\-]+)",
        "Deposito": r"(?i)\b(?:Deposito|Simpanan berjangka)\b\s+([\d.,\-]+)",
        "Uang Elektronik": r"(?i)\bUang Elektronik\b\s+([\d.,\-]+)",
        "Liabilitas kepada Bank Indonesia": r"(?i)\bLiabilitas kepada Bank Indonesia\b\s+([\d.,\-]+)",
        "Liabilitas kepada bank lain": r"(?i)\bLiabilitas kepada bank lain\b\s+([\d.,\-]+)",
        "Liabilitas spot dan derivatif/forward":  r"(?i)\bLiabilitas spot dan derivatif\s*/?\s*forward\b\s*([\d.,\-]+)",
        "Liabilitas atas surat berharga yang dijual dengan janji dibeli kembali (repo)":r"(?i)\bLiabilitas atas surat berharga yang dijual dengan janji dibeli kembali\b\s+\(repo\)\s+([\d.,]+)",
        "Liabilitas akseptasi": r"(?i)\bLiabilitas akseptasi\b\s+([\d.,\-]+)",
        "Surat berharga yang diterbitkan": r"(?i)\bSurat berharga yang diterbitkan\b\s+([\d.,\-]+)",
        "Pinjaman/pembiayaan yang diterima":  r"(?i)\bPinjaman\s*/?\s*pembiayaan yang diterima\b\s*([\d.,\-]+)",
        "Setoran jaminan": r"(?i)\bSetoran jaminan\b\s+([\d.,\-]+)",
        "Liabilitas antarkantor": r"(?i)\bLiabilitas Antarkantor\b.*?([\d.,]+)",
        "Liabilitas lainnya": r"(?i)\bLiabilitas lainnya\b\s+([\d.,\-]+)",
        "Total Liabilitas": r"(?i)\b dont modify this code\b\s+([\d.,]+)",
        "Modal disetor": r"(?i)\b dont modify this code\b\s+([\d.,]+)",
        "a. Modal dasar": r"(?i)\ba\.\s*Modal dasar\b\s+([\d.,]+)",
        "b. Modal yang belum disetor": r"(?i)\bb\.\s*Modal yang belum disetor\s+([\d.,\-]+)",
        "c. Saham yang dibeli kembali (treasury stock)": r"(?i)\bSaham yang dibeli kembali \(treasury stock\)\s*(-?[\d\.,]+)",
        "Tambahan modal disetor": r"(?i)\bDont modify this code\b\s+(-*[\d.,\-]+)",
        "a. Agio": r"(?i)\ba\.\s*Agio\b\s+([\d.,\-]+)",
        "b. Disagio": r"(?i)\bb\.\s*Disagio\b\s+\-\-\s+([\d.,\-]+)",
        "c. Dana setoran modal": r"(?i)\bc\.\s*Dana setoran modal\b\s+(-*[\d.,\-]+)",
        "d. Modal Sumbangan": r"(?i)\bDont modify this code\b\s+(-*[\d.,\-]+)",
        "e. Lainnya": r"(?i)\bd\.\s*Lainnya\b\s+(-*[\d.,\-]+)",
        "Penghasilan komprehensif lain": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "a.Keuntungan": r"(?i)\b(?:a\.?\s*Keuntungan)\b\s+([\d.,]+)",
        "b.Kerugian": r"(?i)\b(?:b\.?\s*Kerugian)\b\s*([\d.,\-]+)",
        "Cadangan": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Cadangan umum": r"(?i)\bCadangan umum\b\s+([\d.,]+)",
        "Cadangan tujuan": r"(?i)\bCadangan tujuan\b\s+(-*[\d.,\-]+)",
        "Laba/rugi": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Tahun-tahun lalu": r"(?i)\ba\.\s*Tahun[-\s]*tahun[-\s]*lalu\b\s+([\d.,]+)",
        "Tahun berjalan": r"(?i)\bTahun\s*berjalan\b\s*(?:(?:\(\d+\)\s*)|(?:\[\d+\]\s*)|(?:\d\))\s*)?([\d.,-]+)",
        "Dividen yang dibayarkan": r"(?i)\bDividen yang dibayarkan\b\s*[-/]*\s*([\d.,\-]+)",
        "Total Ekuitas": r"(?i)\b dont modify this code\b\s+([\d.,]+)",
        "Total Liabilitas & Ekuitas": r"(?i)\b dont modify this code\b\s+([\d.,]+)",
        "Pendapatan dan Beban Bunga": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Pendapatan Bunga": r"(?i)\bPendapatan Bunga\b\s+([\d.,\-]+)",
        "Beban Bunga": r"(?i)^\d+\.\s*Beban Bunga\s+([\d.,\-]+)",
        "Pendapatan dan Beban Operasional lainnya": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Nilai wajar aset keuangan": r"(?i)\bNilai wajar aset keuangan\b\s+([\d.,\-]+)",
        "Nilai wajar liabilitas keuangan": r"(?i)\bNilai wajar liabilitas keuangan\b\s+([\d.,\-]+)",
        "Keuntungan (kerugian) dari penjualan aset keuangan": r"(?i)\bKeuntungan \(kerugian\) dari penjualan aset keuangan\b\s+([\d.,\-]+)",
        "Keuntungan (kerugian) dari transaksi spot dan derivatif/forward (realised)": r"(?i)\d*\.*\s*Keuntungan \(kerugian\) dari transaksi spot\s*(?:dan|\/)?\s*derivatif\s*(?:\/)?\s*forward \(realised\)\s+([\d.,\-]+)",
        "Keuntungan (kerugian) dari penyertaan dengan equity method": r"(?i)\bKeuntungan \(kerugian\) dari penyertaan dengan equity method\b\s+([\d.,\-]+)",
        "Keuntungan (kerugian) dari penjabaran transaksi valuta asing": r"(?i)\bKeuntungan \(kerugian\) dari penjabaran transaksi valuta asing\b\s+([\d.,\-]+)",
        "Pendapatan dividen": r"(?i)\bPendapatan dividen\b\s+([\d.,\-]+)",
        "Pendapatan komisi/provisi/fee dan administrasi": r"(?i)\bkomisi/provisi/fee dan administrasi\b\s+([\d.,\-]+)",
        "Pendapatan lainnya": r"(?i)\bPendapatan lainnya\b\s+([\d.,\-]+)",
        "Beban (pemulihan) kerugian penurunan nilai aset keuangan (impairment)": r"(?i)\d*\.*\s*kerugian penurunan nilai aset keuangan \(impairment\)\s+([\d.,\-]+)",
        "Kerugian terkait risiko operasional": r"(?i)\bKerugian terkait risiko operasional\b\s+([\d.,\-]+)",
        "Beban tenaga kerja": r"(?i)\bBeban tenaga kerja\b\s+([\d.,\-]+)",
        "Beban promosi": r"(?i)\bBeban promosi\b\s+([\d.,\-]+)",
        "Beban lainnya": r"(?i)\bBeban lainnya\b\s+([\d.,\-]+)",
        "Pendapatan (Beban) Operasional Lainnya Bersih": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "LABA (RUGI) OPERASIONAL": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Keuntungan (kerugian) penjualan aset tetap dan inventaris": r"(?i)\bKeuntungan \(kerugian\) penjualan aset tetap dan inventaris\b\s+([\d.,\-]+)",
        "Pendapatan (beban) non operasional lainnya": r"(?i)\bPendapatan \(beban\) non operasional lainnya\b\s+([\d.,\-]+)",
        "LABA (RUGI) NON OPERASIONAL":  r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "LABA (RUGI) PERIODE BERJALAN SEBELUM PAJAK":  r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Pajak Penghasilan":r"(?i)\bDont Modify This Code\b\s+([\d.,]+)",
        "Taksiran pajak periode berjalan": r"(?i)\b(?:a\.|b\.)?\s*Taksiran pajak(?:\s*[a-z]*)?\s*berjalan\s*(?:-\/-\s*)?([\d.,\-]+)",
        "Pendapatan (beban) pajak tangguhan": r"(?i)\b(?:\(Pendapatan\))?\s*\(?Pendapatan\)?\s*\(?beban\)?\s*pajak\s*tangguhan\b\s*(-?[\d.,]+)",
        "LABA (RUGI) BERSIH PERIODE BERJALAN": r"(?i)\bDont Modify This Code\b\s+([\d.,]+)"
    }
    data = []
    for account, pattern in patterns.items():
        matched = False
        for line in cleaned_lines:
            match = re.search(pattern, line)
            if match:
                data.append({"Account": account, "Amount": match.group(1)})
                matched = True
                break
        if not matched:
            data.append({"Account": account, "Amount": 0})
    df = pd.DataFrame(data)
    return df

## Load & Process Data

In [34]:
# Extract PDF
env = 'https://ir.bankbjb.co.id/files//2023/02/lbu-januari-2023-bjbr-2.pdf'
cleaned_lines = process_pdf_from_url(env)
df_result = extract_account_amount(cleaned_lines)

# Cleaning Result
df_result['Amount'] = df_result['Amount'].str.replace('.', ',', regex=False)
df_result['Amount'] = df_result['Amount'].replace('-', '0')
df_result['Amount'] = df_result['Amount'].fillna('0')
df_result['Amount'] = df_result['Amount'].str.replace(',', '').astype(int)

## Define the list of null account
account_list = [
    'Cadangan kerugian penurunan nilai aset keuangan -/-',
    'Aset non produktif',
    'Modal diseter',
    'Penghasilan komprehensif lain',
    'Cadangan',
    'Laba/rugi',
    'Pendapatan dan Beban Bunga',
    'Pendapatan dan Beban Operasional lainnya',
    'Pendapatan (Beban) Operasional Lainnya Bersih',
    'LABA (RUGI) PERIODE BERJALAN SEBELUM PAJAK',
    'LABA (RUGI) OPERASIONAL',
    'LABA (RUGI) NON OPERASIONAL',
    'LABA (RUGI) BERSIH PERIODE BERJALAN',
    'Total Aset',
    'Total Liabilitas',
    'Total Ekuitas',
    'Total Liabilitas & Ekuitas',
    'Pajak Penghasilan',
    'Modal disetor',
    'Tambahan modal disetor'
]
result = df_result.loc[df_result['Account'].isin(account_list), 'Amount'] = ''

# Filter rows where 'Amount' is 0
df_zero_amount = df_result[df_result['Amount'] == 0]
df_zero_amount

## Grouping Account
result = pd.DataFrame()

kelompok = []

for i in range(len(df_result)):
    if i < 27:
        kelompok.append('Aset')
    elif 27 <= i < 42:
        kelompok.append('Liabilitas')
    elif 42 <= i < 63:
        kelompok.append('Ekuitas')
    elif i == 63:
        kelompok.append('Total Liabilitas & Ekuitas')
    else:
        kelompok.append('Laba Rugi')

df_result['kelompok'] = kelompok
df_result = df_result[['kelompok'] + [col for col in df_result.columns if col != 'kelompok']]
df_result[89:91]


## Modify Sign
def modify_amount(row):
    positive_accounts = [
        'Beban Bunga',
        'Beban (pemulihan) kerugian penurunan nilai aset keuangan (impairment)',
        'Kerugian terkait risiko operasional',
        'Beban tenaga kerja',
        'Beban promosi',
        'Beban lainnya',
        'Taksiran pajak periode berjalan'
    ]

    negative_accounts = [
        'a. Surat berharga yang dimiliki',
        'b. Kredit yang diberikan dan pembiayaan syariah',
        'c. Lainnya',
        'Akumulasi amortisasi aset tidak berwujud',
        'Akumulasi penyusutan aset tetap dan inventaris -/-',
        'b. Modal yang belum disetor',
        'c. Saham yang dibeli kembali (treasury stock)',
        'b.Kerugian',
        'Dividen yang dibayarkan'
    ]

    amount_str = str(row['Amount']).replace(',', '').strip()
    try:
        amount = float(amount_str)
    except ValueError:
        amount = 0 
        # If account is in positive accounts
    if row['Account'] in positive_accounts:
        # Ensure the amount is positive
        if amount < 0:
            return int(-amount)  # Multiply by -1 and convert to int
        else:
            return int(amount)

    # If account is in negative accounts
    elif row['Account'] in negative_accounts:
        # Ensure the amount is negative
        if amount > 0:
            return int(-amount)  # Multiply by -1 and convert to int
        else:
            return int(amount)

    # If account is neither positive nor negative, keep amount unchanged
    else:
        return int(amount)


df_result['Modified_Amount'] = df_result.apply(modify_amount, axis=1)
df_result['Modified_Amount'] = df_result['Modified_Amount'].astype(int)

# Export the DataFrame to an Excel file
converter_result = "financial_report_extracted.xlsx"
df_result.to_excel(converter_result, index=False)
